In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import scipy
import time
import calendar, datetime, pytz
import csv
import pandas
import copy
import pickle
import math

from IPython.display import display, HTML

from sklearn.externals import joblib
from sklearn.pipeline import make_pipeline, make_union
import sklearn.linear_model
import sklearn.dummy
import sklearn.metrics
import sklearn.ensemble
import sklearn.calibration
#import xgboost
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm_notebook
import tqdm

%matplotlib inline

import seaborn as sns
def setSnsStyle(style):
    sns.set(style=style, font_scale=1.5)
    font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 30}
    matplotlib.rc('font', **font)
    matplotlib.rcParams['xtick.labelsize'] = 16 
    matplotlib.rcParams['ytick.labelsize'] = 16
    matplotlib.rcParams['axes.titlesize'] = 16
    matplotlib.rcParams['axes.labelsize'] = 16
    
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
%config InlineBackend.figure_format = 'retina'

setSnsStyle("ticks")

In [3]:
%load_ext autoreload
%autoreload 1
%aimport bb_behavior
%aimport bb_behavior.io
%aimport bb_behavior.db
%aimport bb_behavior.db.sampling
%aimport bb_behavior.db.trajectory
%aimport bb_behavior.io.videos
%aimport bb_behavior.io.biotracker
%aimport bb_behavior.plot.videos
%aimport bb_behavior.trophallaxis.prefilter
%aimport bb_behavior.trophallaxis.trajectory_filter

import bb_behavior
import bb_behavior.db
import bb_behavior.db.sampling
import bb_behavior.db.trajectory
import bb_behavior.io
import bb_behavior.io.videos
import bb_behavior.plot.videos
import bb_behavior.io.biotracker
import bb_behavior.trophallaxis.prefilter
import bb_behavior.trophallaxis.trajectory_filter

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


SyntaxError: invalid syntax (videos.py, line 35)

In [4]:
import pytz

In [5]:
with open('/home/mi/rrszynka/mnt/janek/caches/Other/bee_exits_timestamps.pkl', 'rb') as f:
    bee_exits_dict = pickle.load(f)

#bee_id = 885
#timestamps = bee_exits_dict[bee_id][:3]

In [6]:
def to_datetime(ts):
    return pytz.UTC.localize(pandas.Timestamp(ts).to_pydatetime(), pytz.UTC)

In [7]:
import bb_behavior.db.sampling

In [8]:
from concurrent.futures import ProcessPoolExecutor

In [9]:
def get_data_for_bee_id_timestamps(args):
    bee_id, timestamps = args
    timestamps = [to_datetime(ts) for ts in timestamps]
    bee_df = None
    
    with bb_behavior.db.DatabaseCursorContext() as cursor:
        cursor.execute("PREPARE fetch_cam_id AS "\
                       "SELECT cam_id FROM bb_detections_2016_stitched "
                       "WHERE bee_id = $1 "
                       "AND timestamp >= $2 and timestamp <= $3 "
                       "LIMIT 1")
        
        def fetch_cam_id(bee_id, ts, cursor):
            cursor.execute("EXECUTE fetch_cam_id(%s, %s, %s)", (bee_id, ts - offset, ts))
            data = cursor.fetchone()
            if data is None:
                return None
            return data[0]
        
        def fetch_cam_id_(ts):
            return fetch_cam_id(bee_id, ts, cursor)
        cam_ids = list(map(fetch_cam_id_, timestamps))
        
        bee_df = list(zip([bee_id] * len(timestamps), timestamps, cam_ids))
                
        for ts_idx, (bee_id, ts, cam_id) in enumerate(bee_df):
            
            posix_ts_from = (ts-offset).timestamp()
            posix_ts_to = ts.timestamp()
            x, y = None, None
            if cam_id is not None:
                frames = bb_behavior.db.sampling.get_frames(cam_id, posix_ts_from, posix_ts_to,
                                                            cursor=cursor, cursor_is_prepared=True)
                traj = bb_behavior.db.trajectory.get_bee_detections(bee_id, frames=frames,
                                                                    cursor=cursor, cursor_is_prepared=True,
                                                                    use_hive_coords=True)
                xy = [t[2:4] for t in traj if t is not None]
                if len(xy) > 0:
                    xy = np.array(xy)
                    try:
                        x, y = xy.mean(axis=0)
                    except Exception as e:
                        print(str(e))
                        print(xy)
            bee_df[ts_idx] = (bee_id, ts, x, y, cam_id)
    return bee_df

In [10]:
#subset = dict()
#subset[bee_id] = bee_exits_dict[bee_id]
subset = bee_exits_dict
offset = datetime.timedelta(seconds=15)
all_data = []

with ProcessPoolExecutor(max_workers=16) as pool:
    items = subset.items()
    
    for sub_df in tqdm_notebook(pool.map(get_data_for_bee_id_timestamps, items), total=len(items)):
        all_data += sub_df
    #frames = bb_behavior.db.sampling.get_neighbour_frames(frame_id=None, n_frames_left=10, n_frames_right=0,

Process Process-11:
Process Process-1:
Process Process-5:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-3:
Process Process-12:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Process Process-10:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
Process Process-8:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
 

KeyboardInterrupt: 

Process Process-14:
Process Process-6:
Process Process-16:
Process Process-13:
Process Process-7:
Process Process-2:
Traceback (most recent call last):
Process Process-15:
Traceback (most recent call last):
Process Process-9:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/

In [ ]:
all_data_df = pandas.read_hdf("/home/mi/rrszynka/mnt/janek/caches/Other/with_coordinates.hdf5")

In [ ]:
from collections import defaultdict

In [ ]:
skipped = 0
hists = defaultdict(list)
bins = np.linspace(0, 400, 100)
for hive_side, side_df in all_data_df.groupby("hive_side"):
    if hive_side == 2:
        continue
        
    fig, (ax, ax2) = plt.subplots(2, 1, figsize=(20, 20))    
    n_colors = len(side_df.bee_id.unique())
    cmap = matplotlib.cm.hsv
    
    for bee_idx, (bee_id, bee_df) in tqdm_notebook(enumerate(side_df.groupby("bee_id"))):
        xy = bee_df[["x", "y"]].values
        if xy.shape[0] < 100 or bee_idx % 2 != 0:
            skipped += 1
            continue
        xy = xy[::3, :]
        color = cmap(bee_idx / n_colors)
        
        ax.scatter(xy[:, 0], xy[:, 1], marker="x", c=color, alpha=0.2)
        origin = np.array([0, 250]) if hive_side == 1 else np.array([350, 250])
        xy = xy[~np.any(np.isnan(xy), axis=1), :]
        xy = xy - origin
        distance = np.linalg.norm(xy, axis=1)
        sns.distplot(distance, ax=ax2, norm_hist=False, bins=bins, hist=False, kde=True)
        h, _ = np.histogram(distance, bins=bins)
        hists[hive_side].append(h)
        #if bee_idx > 10:
        #    break
    ax2.set_xlim(0, max(bins))
    #ax2.set_ylim(0, 100)
    ax.set_title("Hive_side: {}".format(hive_side))
    plt.show()
print("Skipped: {}".format(skipped))
#hists = np.array(hists)
#hists.shape

In [ ]:
import scipy.ndimage

In [ ]:
h = np.vstack(hists[1]).astype(np.float32)
h = scipy.ndimage.gaussian_filter1d(h, 10, 0)

In [ ]:
h.shape

In [ ]:
hists_df = pandas.DataFrame(data=h)
hists_df.columns = bins.astype(np.int)[:-1]
sns.clustermap(hists_df, col_cluster=False, z_score=None, figsize=(20,20), standard_scale=0)